
# 형태소 분석 기반 토큰화의 문제
- 형태소 분석기는 작성된 알고리즘 또는 학습된 내용을 바탕으로 토큰화를 하기 때문에 오탈자나 띄어쓰기 실수, 신조어, 외래어, 고유어 등이 사용된 경우 제대로 토큰화 하지 못한다.
- 그래서 발생 할 수있는 잠재적 문제점
    - 어휘사전을 크게 만든다.
        - 같은 의미의 단어가 형태소 분석이 안되어 여러개 등록될 수있다.
        - ex) 신조어 `돈쭐` 이라는 단어를 인식 못할 경우 `"돈쭐내러", "돈쭐나", "돈쭐냄"` 등이 다 등록 될 수 있다.
    - OOV(Out Of Vocab)에 대응하기 어렵게 만든다.
        - 같은 어근의 단어가 있지만 조사등이 바뀐 신조어등을 OOV로 인식할 수있다.




> ### 어휘 사전(Vocabulary)과 Out Of Vocabulary (OOV)
> 
> - 언어 모델링에서 **어휘 사전(Vocabulary)**은 모델이 처리할 수 있는 단어(토큰)들의 집합이다.  
> - 어휘 사전은 보통 전체 데이터셋을 토큰화한 후, 각 토큰을 고유한 정수 인덱스로 매핑해 만든다.
>    - 매핑된 정수는 모델에 입력되는 텍스트 데이터를 숫자 형식으로 변환해 모델이 처리할 수 있도록 돕는다.
>    - 예시) {"I": 1, "he": 2, "you": 3, ...}
> - **Out Of Vocabulary (OOV)**
>    - 어휘 사전(Vocab): 코퍼스를 구성하는 모든 토큰의 집합.
>    - **OOV**란 어휘 사전에 포함되지 않은 토큰을 의미하며, 모델이 해당 토큰을 처리할 수 없기 때문에 일반적으로 특별한 토큰(예: `[UNK]`)으로 대체되거나 다른 방식으로 처리된다.



# Subword Tokenization(하위 단어 토큰화)

## 정의

- Subword Tokenization은 단어를 더 작은 단위(subword)로 나누어 텍스트를 토큰화하는 방식이다.  
    - subword는 하나의 단어를 구성하는 단어들을 말한다.(coworker: co, work, er)
- 주로 자주 등장하는 단어의 일부를 공통된 토큰으로 만들고, 희귀하거나 복합적인 단어는 작은 조각(subword)으로 나누어 처리한다.
- 단어 자체를 그대로 사용하기보다는 단어의 일부를 나누어 처리함으로써 새로운 단어나 미등록 단어(Out-of-Vocabulary) 문제를 줄일 수 있다.

## 장점

1. **미등록 단어 처리 가능**  
   -  새로운 단어(신조어, 속어, 고유어등)가 등장해도 미리 정의된 subword를 조합해서 표현할 수 있어 OOV 문제를 줄일 수 있다.  

2. **어휘 크기 축소**  
   - 같은 subword를 여러 단어에서 공유함으로써, 완전한 단어를 사용하는 경우보다 어휘집의 크기를 작게 유지할 수 있다.


## 종류

1. **Byte-Pair Encoding (BPE)**  
   - 자주 등장하는 문자 쌍을 반복적으로 병합해 서브워드를 생성하는 방식.
   - OpenAI의 GPT 모델에 사용된 토크나이저이다.
   - 각 어절 단위로 분리 후, 조합 (빈도수)

2. **Unigram**  
   - 빈도기반 확률모델에 따라 subword 단위를 선택하는 방식이다.  
   - BPE보다 유연하여 더 다양한 분할 결과를 얻을 수 있다.
   - 모든 경우의수 조합 후, 확률이 낮은 것부터 제거 

3. **WordPiece**  
   - BPE와 유사하지만, 빈도수가 아니라, 가능성이 높은 조합(합쳐질 가능성이 높은 subword)에 기반해 subword들을 찾는다.
   - Google의 BERT 모델에 사용된 토크나이저이다.
   - 각 어절 단위로 분리, 조합 (확률개념)

# Byte Pair Encoding 방식

- 원래 Text data 압축을 위해 만들어진 방법으로 text 에서 많이 등장하는 두글자 쌍의 조합을 찾아 부호화하는 알고리즘이다. 
- 연속된 글자 쌍이 더 나타나지 않거나 정해진 어휘사전 크기에 도달 할 때 까지 조합을 찾아 부호화 하는 작업을 반복한다.

## text 압축 방식의 예
- 원문: abracadabra
1. AracadAra: ab -> A :=> 원문에서 가장 빈도수 많은 ab를 A(부호로 아무 글자나 사용할 수 있다.)로 치환
2. ABcadAB: ra -> B :=> 1에서 가장 빈도수가 많은 ra를 B로 치환
3. CcadC: AB -> C :=> 2에서 가장 빈도수 맣은 AB를 C로 치환한다.(치환된 글자 쌍도 변환대상에 포함된다.)

## BPE Tokenizer 방식
BPE 토크나이저는 자주 등장하는 글자 쌍을 찾아 치환하는 대신 **단어 사전**에 추가한다.

### 예)
1. 말뭉치의 토큰들의 빈도수, 어휘사전은 아래와 같을 경우
    - 빈도사전: ('low', 5), ('lower', 2), ('newest', 6), ('widest', 3)
    - 어휘사전: ['low', 'lower', 'newest', 'widest']
2. 빈도 사전내의 모든 단어들을 글자 단위로 나눈다. (Pre Tokenization)
    - 빈도사전: ('l', 'o', 'w',  5), ('l', 'o', 'w', 'e', 'r', 2), ('n', 'e', 'w', 'e', 's', 't', 6), ('w', 'i', 'd', 'e', 's', 't', 3)
    - 어휘사전: ['d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w']
3. 빈도 사전을 기준으로 가장 자주 등장하는 글자 쌍(byte pair)를 찾는다.  위에서는 **'e'와 's'가 총 9번으로 가장 많이 등장함**. 'e'와 's'를 'es'로 합치고 어휘 사전에 추가한다.
    - 빈도사전: ('l', 'o', 'w',  5), ('l', 'o', 'w', 'e', 'r', 2), ('n', 'e', 'w', **'es'**, 't', 6), ('w', 'i', 'd', **'es'**, 't', 3)
    - 어휘사전: ['d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w', **'es'**]
4. 3 번의 과정을 계속 반복한다. 빈도수가 가장 많은 'es'와 't' 쌍을 'est'로 병합하고 'est'를 어휘 사전에 추가한다.
    - 빈도사전: ('l', 'o', 'w',  5), ('l', 'o', 'w', 'e', 'r', 2), ('n', 'e', 'w', **'est'**, 6), ('w', 'i', 'd', **'est'**, 3)
    - 어휘사전: ['d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w', **'es'**, **'est'**]
5. 만약 10번 반복했다고 하면 다음과 같은 빈도 사전과 어휘 사전이 생성된다.
    - 빈도 사전: (**'low'**, 5), (**'low'**, 'e', 'r', 2), ('n', 'e', 'w', **'est'**, 6), ('w', 'i', 'd', **'est'**, 3)
    - 어휘사전: ['d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w', **'es'**, **'est'**, **'lo'**,**'low'**, **'low'**, **'ne'**, **'new'**, **'newest'**, **'wi'**, **'wid'**, **'widest'**]

- 위와 같이 어휘 사전이 만들어 지면 원래 어휘서전에 없던 것들에 대한 처리를 할 수있다.
    - ex)
        - 'newer' :=> 'new', 'e', 'r', 
        - 'lowest' :=> 'low', 'est'
        - 'wider' :=> 'wid', 'e', 'r'

# WordPiece tokenizer

- Byte Pair Encoding 이 빈도 기반이라면 wordpiece tokenizer는 확률 기반으로 글자 쌍을 병합한다.
- 두개 글자 쌍의 빈도수를 각 개별 글자 빈도수의 곱으로 나눈 점수가 가장 높은 순서대로 글자쌍을 묶어 나간다.

$$
score = \cfrac{f(x, y)}{f(x)\cdot f(y)} 
$$

함수 f는 빈도를 나타내며 x, y는 병합하려는 하위 단어이다.

- 빈도사전: ('l','o','w', 5), ('l','o','w', 'e', 'r', 2), ('n', 'e', 'w', 'e', 's', 't', 6), ('w', 'i', 'd', 'e', 's', 't', 3)
- 어휘사전: ('d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w')
- 가장 빈도수가 높은 쌍은 'e','s'로 9번 등장한다. 이때 각 글자는 전체에서 각각 'e'는 17번, 's'는 9번 등장한다. 위 공식에 대입하면 score는 $\frac{9}{17 \times 9} \approx 0.06$ 이다.
- 'i'와 'd' 쌍은 3번만 등장하지만 전체에서 각각 'i' 3번, 'd' 3번 등장한다. 그래서 score는 $\frac{3}{3 \times 3} \approx 0.33$ 이다.
- 나타난 빈도수는 'es' 가 많치만 더 높은 score를 가지는 'id' 쌍을 병합한다.
- 빈도사전: ('l','o','w', 5), ('l','o','w', 'e', 'r', 2), ('n', 'e', 'w', 'e', 's', 't', 6), ('w', **'id'**, 'e', 's', 't', 3)
- 어휘사전: ('d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w', **'id'**)
위의 작업을 반복해 연속된 글자 쌍이 더이상 나타나지 않거나 어휘 사전 max 크기에 도달할 때 까지 학습한다.

# Unigram 방식
- 빈도 기반 확률 모델을 사용하여 효율적으로 서브워드를 선택하고, 불필요한 서브워드를 제거해 최적의 어휘 크기를 찾는 알고리즘


- **초기 어휘 집합 구성**
    - 대상 text에 모든 단어와 그 서브스트링을 포함한 어휘 집합을 생성한다. 이 어휘 집합은 나올 수있는 모든 subword들을 다 모아놓은 것이다. 
    - 예를 들어 "hug" 단어의  ["h", "u", "g", "hu", "ug", "hug"]  substring을 만든다. 이들이 subword 후보가 된다.
- **각 Subword의 빈도수 기반 확률 계산**
    -  $\cfrac{subword가\;나타난\;횟수}{전체\;빈도수}$ 로 각 subword들의 나타난 확률을 계산한다.
- **가능한 분할에 대한 확률 계산**
    - 단어를 여러 서브워드로 분할할 수 있는 경우, 각 분할에 대한 전체 확률을 계산한다.
    - 확률 계산은 $ P(subword1)\;\times \; P(subword2)\;\times\; ..$ 으로 계산한다.
    - 예를 들어 "hug" 를 분할 한다고 했을 때
        1. \["h", "u", "g"\]: $ P(h) \times P(u) \times P(g) $
        2. \["hu", "g"\]: $ P(pu) \times P(g) $

   - 각각의 확률을 계산한 후, **가장 높은 확률**을 가진 분할을 선택한다.
     - 위 예에서 만약 1의 확률이 0.01 이고 2의 확률이 0.00001 이라면 첫번째 분할이 선택된다.

- **서브워드 제거**
    - 위의 훈견과정에서 불필요한 서브워드를 제거하면서 최적의 어휘 집합을 찾아간다. 
    - 제거 대상은 빈도수가 낮거나 조합에 크게 영향을 주지 않은 subword들이다.

In [4]:
from Korpora import Korpora

corpus = Korpora.load("korean_petitions")


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : Hyunjoong Kim lovit@github
    Repository : https://github.com/lovit/petitions_archive
    References :

    청와대 국민청원 게시판의 데이터를 월별로 수집한 것입니다.
    청원은 게시판에 글을 올린 뒤, 한달 간 청원이 진행됩니다.
    수집되는 데이터는 청원종료가 된 이후의 데이터이며, 청원 내 댓글은 수집되지 않습니다.
    단 청원의 동의 개수는 수집됩니다.
    자세한 내용은 위의 repository를 참고하세요.

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/



[korean_petitions] download petitions_2017-08: 1.84MB [00:00, 3.08MB/s]                            
[korean_petitions] download petitions_2017-09: 20.4MB [00:02, 9.99MB/s]                            
[korean_petitions] download petitions_2017-10: 12.0MB [00:03, 3.35MB/s]                            
[korean_petitions] download petitions_2017-11: 28.4MB [00:01, 23.9MB/s]                            
[korean_petitions] download petitions_2017-12: 29.0MB [00:01, 23.1MB/s]                            
[korean_petitions] download petitions_2018-01: 43.9MB [00:17, 2.58MB/s]                            
[korean_petitions] download petitions_2018-02: 33.8MB [00:21, 1.58MB/s]                            
[korean_petitions] download petitions_2018-03: 34.3MB [00:18, 1.88MB/s]                            
[korean_petitions] download petitions_2018-04: 35.5MB [00:09, 3.56MB/s]                            
[korean_petitions] download petitions_2018-05: 37.5MB [00:16, 2.31MB/s]                            


In [5]:
petitions = corpus.get_all_texts() # 청원 데이터를 반환. list[str] str: 개별 청원
len(petitions)

433631

In [6]:
print(petitions[0])

안녕하세요. 현재 사대, 교대 등 교원양성학교들의 예비교사들이 임용절벽에 매우 힘들어 하고 있는 줄로 압니다. 정부 부처에서는 영양사의 영양'교사'화, 폭발적인 영양'교사' 채용, 기간제 교사, 영전강, 스강의 무기계약직화가 그들의 임용 절벽과는 전혀 무관한 일이라고 주장하고 있지만 조금만 생각해보면 전혀 설득력 없는 말이라고 생각합니다. 학교 수가 같고, 학생 수가 동일한데 영양교사와 기간제 교사, 영전강 스강이 학교에 늘어나게 되면 당연히 정규 교원의 수는 줄어들게 되지 않겠습니까? 기간제 교사, 영전강, 스강의 무기계약직화, 정규직화 꼭 전면 백지화해주십시오. 백년대계인 국가의 교육에 달린 문제입니다. 단순히 대통령님의 일자리 공약, 81만개 일자리 창출 공약을 지키시고자 돌이킬 수 없는 실수는 하지 않으시길 바랍니다. 세계 어느 나라와 비교해도, 한국 교원의 수준과 질은 최고 수준입니다. 고등교육을 받고 어려운 국가 고시를 통과해야만 대한민국 공립 학교의 교단에 설 수 있고, 이러한 과정이 힘들기는 하지만 교원들이 교육자로서의 사명감과 자부심을 갖고 교육하게 되는 원동력이기도 합니다. 자격도 없는 비정규 인력들을 일자리 늘리기 명목 하에 학교로 들이게 되면, 그들이 무슨 낯으로 대한민국이 '공정한 사회' 라고 아이들에게 가르칠 수 있겠습니까? 그들이 가르치는 것을 학부모와 학생들이 납득할 수 있겠으며, 학생들은 공부를 열심히 해야하는 이유를 찾을 수나 있겠습니까? 열심히 안 해도 떼 쓰면 되는 세상이라고 생각하지 않겠습니까? 영양사의 영양교사화도 재고해주십시오. 영양사분들 정말 너무나 고마운 분들입니다. 학생들의 건강과 영양? 당연히 성장기에 있는 아이들에게 필수적이고 중요한 문제입니다. 하지만 이들이 왜 교사입니까. 유래를 찾아 볼 수 없는 영양사의 '교사'화. 정말 대통령님이 생각하신 아이디어라고 믿기 싫을 정도로 납득하기 어렵습니다. 중등은 실과교과 교사가 존재하지요? 초등 역시 임용 시험에 실과가 포함돼 있으며 학교 현장에서도 정규 교원이 직접 

In [8]:
# 파일로 저장
save_petitions_path = "./data/petitions_corpus.txt"
with open(save_petitions_path, 'wt', encoding = 'utf-8') as fw:
    for p in petitions:
        fw.write(p + "\n")

In [9]:
# 저장된 txt 읽기
with open(save_petitions_path, 'rt', encoding = 'utf-8') as fr:
    petitions_txt = fr.read()

# Hugging Face tokenizers 패키지 사용해 토큰화 수행
- Subword tokenization을 처리하는 다양한 패키지(라이브러리)가 있다.

## 주요 라이브러리 
- [tokenizers](https://huggingface.co/docs/tokenizers/index)
    - huggingface에서 개발한 tokenizer 라이브러리로 BPE, WordPiece, Unigram 알고리즘을 지원한다. 
    - 설치: `pip install tokenizers`
- [Sentencepiece](https://github.com/google/sentencepiece)
    - 구글에서 개발한 subword tokenizer 라이브러리로 BPE, Unigram 방식 지원.
    - 설치: `pip install sentencepiece`

> ### korpora 말뭉치
> - 다양한 한글 데이터셋을 제공하는 패키지
> - `pip install korpora`

## Hugging Face tokenizers 패키지이용

- 설치: `pip install tokenizers`
- Tokenizer 생성
    - 토큰화 알고리즘을 지정해 instance 생성.
- Trainer 생성
    - 학습 파라미터를 설정해서 instance 생성
- Tokenizer 학습
    - train() 메소드: 학습 text 파일 경로를 지정해서 학습
    - train_from_iterator() 메소드: 학습할 string들을 iterator를 통해 제공.
- https://github.com/huggingface/tokenizers



In [ ]:
import time
# Tokenizer
from tokenizers import Tokenizer 
from tokenizers.models import BPE
# subword 알고리즘을 적용하기 전에 어떻게 나눠놓을 것인지.
from tokenizers.pre_tokenizers import Whitespace
# Trainer (학습)
from tokenizers.trainers import BpeTrainer

# 토크나이저 생성
# subword 알고리즘을 구현한 Tokenizer의 객체를 넣어 생성. (BPE)
# unk_token : OOV(out of voc)(unknown) 단어(토큰)을 처리할 토큰을 지정.
tokenizer = Tokenizer(BPE(unk_token= '[UNK]'))
# pre tokenizer를 등록
tokenizer.pre_tokenizer = Whitespace() # 공백을 기준으로 미리 토큰화하여 놓는다.
# tokenzer를 학습하는 Trainer 객체 -> tokenizer 알고리즘 별로 Trainer클래스가 제공됨.
# initalizer에 어떻게 학습시킬지 설정
trainer = BpeTrainer(
    vacab_size = 10000, # 어휘사전의 최대 크기(고유 토큰의 최대 갯수) / 보통 30000개 정도 설정
    min_frequency = 10, 
    # 사전에 넣을 토큰의 **최소 출연 횟수**(빈도수) -> 어떤 토큰을 사전에 등록 할 때 n번 이상 출현해야 사전에 등록 ex) ab / 5번 -> 사전미등록
    special_tokens = ["[UNK],", "[PAD]"]
    # 어휘사전에 추가할 특수토큰(special token)들 지정. 이중 unk_token은 반드시 설정.

)

# special token : (토큰이름은 자율적으로 지정해도 상관없지만 필요한 토큰을 용도에 맞게 사용)
# [UNK] - oov 토큰을 표시. 
# [PAD] - 문장의 토큰수를 맞추기 위한 padding.(자릿수 맞추기 위한 용도)
# [CLS] - 문장의 시작을 표시 + 전체 문장의 의미를 저장하는 토큰으로 사용(BERT모델에서 주로 사용).
# [SOS] - 문서의 시작을 표시.
# [EOS] - 문서의 끝을 표시.
# [SEP] - 문서가 여러 문장으로 구성된 경우 문장을 구분하는 용도.
# [MASK] - 일부 토큰을 가리는 토큰.(기존 토큰을 대체하여 삽입.) -> LLM 모델중 감춰진 부분을 맞춰가며 학습하는 모델이 존재.

# [PAD] 예시 / 입력데이터 - 5개의 토큰으로 통일 -> 4개 이하의 토큰은 [PAD] 토큰으로 채우고, 6개 이상은 뒷 토큰들 제거하는 방식으로 구성

In [52]:
# 학습 - corpus를 받아서 어휘사전을 만드는 작업.
s = time.time()
tokenizer.train(["data/petitions_corpus.txt"], trainer=trainer) # 학습시킬 파일 경로(들), trainer 객체
e = time.time()
print("걸린 시간 : ", (e-s), "초")

걸린 시간 :  114.5571174621582 초


In [53]:
# 학습된 tokenizer를 disk에 저장.
import os
os.makedirs("saved_models", exist_ok = True)

# 저장 - tokenizer.save(경로)
save_path = "saved_models/petitions_bpe.json"
tokenizer.save(save_path)

In [54]:
# 저장된 모델 load
load_tokenizer = Tokenizer.from_file(save_path)

In [55]:
# vocab size
print("vocab size(어휘사전에 어휘수) : ", load_tokenizer.get_vocab_size())

vocab size(어휘사전에 어휘수) :  30000


In [56]:
tokenizer.get_vocab() # 어휘사전을 dictionary로 반환(key : 단어, value : id)

{'뗫': 4853,
 '때문이라고': 18525,
 '자라는': 16117,
 '기사입니다': 25398,
 '취지와': 28488,
 '신분증': 17675,
 '廷': 1969,
 '객': 3926,
 'ta': 19435,
 '올해': 9678,
 '맞고': 13561,
 '4년제': 20956,
 '죽음에': 24577,
 '벤츠': 27279,
 '전기': 9353,
 '跆': 3480,
 '类': 2994,
 '해수': 26370,
 '보았을': 26595,
 '전수조사를': 28970,
 '117': 29082,
 '가상화폐': 9178,
 '힘없고': 19967,
 '난민으로': 28832,
 '평에': 29613,
 '대놓고': 14817,
 '擲': 2231,
 '누락': 17079,
 '‟': 472,
 '2기': 28814,
 '이은': 22033,
 '싫': 5921,
 '室': 1813,
 '누구는': 14375,
 '만도': 19193,
 ')"': 27553,
 '新': 2271,
 '흡수': 21789,
 '좋겠다': 17120,
 '휙': 7612,
 '낮고': 24629,
 '猾': 2725,
 '숙여': 29778,
 '앛': 6091,
 '룬': 5068,
 '뇔': 4481,
 '선수들에게': 21667,
 '핷': 7449,
 'ろ': 970,
 '迹': 3545,
 '했으나': 11005,
 '얼어': 27236,
 '책임지지': 27071,
 '몡': 5237,
 '소득분위': 29283,
 '피해자들의': 18793,
 'べ': 950,
 '강남구': 27189,
 '우체국': 21308,
 '정보를': 10520,
 '이런식으로': 12348,
 '껏': 4211,
 '北': 1494,
 '렺': 5020,
 '따른': 9431,
 '외국의': 20927,
 '❻': 821,
 '사립유치원': 14503,
 '시점에': 14937,
 '비판하는': 26306,
 '우방': 28473,
 '인고의': 29940,

In [48]:
# 테스트 문장
sports_txt = "프리미어리그 역대 개인 최다골 기록을 보유하고 있는 시어러가 손흥민의 골 결정력을 재차 극찬했다."
petition_txt = "이 글을 쓴 이유는 다름아닌 '전안법'시행 반대를 주장하기 위해서입니다. 먼저, '전안법'은 전기용품 및 생활용품을 판매하는 업체에서 KC인증마크를 의무적으로 받는 것입니다."
comment_txt = "멋진 식사를 즐기기에 좋은 장소 - 채식 메뉴가 정말 훌륭했습니다. 당근 케이크는 아마도 내가 먹어본 디저트 중 최고였을 거예요."

In [49]:
# text -> token 화
token_output = tokenizer.encode(sports_txt)
token_output

Encoding(num_tokens=25, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [ ]:
# 토큰 - 단어
token_output.tokens

['프리미',
 '어리',
 '그',
 '역대',
 '개인',
 '최',
 '다',
 '골',
 '기록을',
 '보유하고',
 '있는',
 '시',
 '어',
 '러',
 '가',
 '손흥민',
 '의',
 '골',
 '결정',
 '력을',
 '재차',
 '극',
 '찬',
 '했다',
 '.']

In [39]:
# 토큰 - 토큰 id(정수)
token_output.ids

[17894,
 11085,
 4128,
 14043,
 8414,
 6891,
 4563,
 4027,
 16508,
 17816,
 8219,
 5908,
 6140,
 4980,
 3902,
 14807,
 6384,
 4027,
 8940,
 8644,
 22240,
 4129,
 6795,
 8565,
 15]

In [57]:
output = tokenizer.encode(petition_txt)
print(output.tokens)

['이', '글을', '쓴', '이유는', '다름', '아닌', "'", '전', '안', '법', "'", '시행', '반대를', '주장', '하기', '위해서', '입니다', '.', '먼저', ',', "'", '전', '안', '법', "'", '은', '전기', '용품', '및', '생활', '용', '품을', '판매하는', '업체에서', 'K', 'C', '인증', '마크', '를', '의무적으로', '받는', '것입니다', '.']


In [58]:
print(output.ids)

[6396, 8577, 6044, 9246, 10139, 8323, 8, 6476, 6073, 5412, 8, 8656, 16307, 8953, 8301, 8580, 8212, 15, 8855, 13, 8, 6476, 6073, 5412, 8, 6360, 9353, 15879, 5345, 8437, 6278, 11250, 18121, 15638, 44, 36, 10497, 16702, 5101, 13014, 8546, 8299, 15]


In [ ]:
# offsets[토큰리스트의 index]
output.offsets[3] # 3번 index : ('이유는', 9246) -> 입력 문서(string)에 어디에 위치하는지 반환. 7번 index ~ 9번 index

(7, 10)

In [ ]:
tokenizer.token_to_id("이유") # 입력값이 몇번째 토큰인지 반환.
tokenizer.id_to_token(8341) # 입력값에 해당하는 id의 토큰내용 반환.

'이유'

In [ ]:
ids = [100, 8231, 8341, 20, 700, 1238]
decode_output = tokenizer.decode(ids) # 토큰 id로 구성된 리스트를 문장(str)으로 decode.
decode_output

'¥ 니까 이유 3 ▶ 什'

In [66]:
tokenizer.decode(output.ids)

"이 글을 쓴 이유는 다름 아닌 ' 전 안 법 ' 시행 반대를 주장 하기 위해서 입니다 . 먼저 , ' 전 안 법 ' 은 전기 용품 및 생활 용 품을 판매하는 업체에서 K C 인증 마크 를 의무적으로 받는 것입니다 ."

## Word Piece 모델 학습

In [1]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import WordPieceTrainer

# tokenizer 생성
tokenizer2 = Tokenizer(WordPiece(unk_token='[UNK]'))
# Pre tokenizer
tokenizer2.pre_tokenizers = Whitespace
# Trainer 생성
trainer = WordPieceTrainer(
    vocab_size=20000,
    special_tokens=['[UNK]', '[PAD]', '[SEP]', '[SOS]', '[EOS]'],
    min_frequency=10
)

In [ ]:
import time
# 학습
s = time.time()
tokenizer2.train(['data/petitions_corpus.txt'],trainer=trainer)
e = time.time()
print(f"걸린 시간 : {e-s}초")

In [ ]:
tokenizer2.save("saved_models/petitions_wordpiece.json")

In [ ]:
load_tokenizer2 = Tokenizer.from_file("saved_models/petitions_wordpiece.json")

In [ ]:
tokenizer2.get_vocab_size()

In [ ]:
tokenizer2.token_to_id('안녕하세요')

In [ ]:
tokenizer2.id_to_token('15453')

In [ ]:
output = tokenizer2.encode(comment_txt)

In [ ]:
output.ids

In [ ]:
output.tokens

In [ ]:
tokenizer2.decode(output.ids)

In [ ]:
comment_txt

In [ ]:
## Unigram 모델 학습

In [3]:
from tokenizers import Tokenizer
from tokenizers.models import Unigram
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import UnigramTrainer

# tokenizer 생성
tokenizer3 = Tokenizer(Unigram())
# Pre tokenizer
tokenizer3.pre_tokenizers = Whitespace
# Trainer 생성
trainer = UnigramTrainer(
    vocab_size=10000,
    special_tokens=['[UNK]', '[PAD]', '[SEP]', '[SOS]', '[EOS]'],
    min_frequency=10
)

In [4]:
import time
# 학습
s = time.time()
tokenizer3.train(['data/petitions_corpus.txt'],trainer=trainer)
e = time.time()
print(f"걸린 시간 : {e-s}초")

: 

In [ ]:
tokenizer3.save("saved_models/petitions_unigram.json")

In [ ]:
tokenizer.token_to_id('안녕하세요')

In [ ]:
tokenizer3.id_to_token()

In [ ]:
output = tokenizer3.encode(comment_txt)
output.tokens

In [ ]:
output.ids

In [ ]:
output.tokens

In [ ]:
tokenizer3.decode(output.ids)
# " ".join(output.tokens)